<a href="https://colab.research.google.com/github/pezkov/gpu_course/blob/main/OpenCLExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!apt install pocl-opencl-icd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pocl-opencl-icd is already the newest version (1.8-3).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [26]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.26)
project(examples_ocl)

set(CMAKE_CXX_STANDARD 17)

find_package(OpenCL REQUIRED)

add_executable(${PROJECT_NAME} example.cpp)

target_link_libraries(${PROJECT_NAME} PUBLIC OpenCL)


Overwriting CMakeLists.txt


In [54]:
%%writefile example.cpp

#include "CL/opencl.hpp"

#include <iostream>
#include <vector>

int main(){
    // Исходные данные
    std::vector<float> A = {1, 2, 3, 4, 5};
    std::vector<float> B = {10, 20, 30, 40, 50};
    std::vector<float> C(A.size());

    // Получаем платформы и устройства
    std::vector<cl::Platform> platforms;
    cl::Platform::get(&platforms);
    cl::Device device;
    for (const auto &platform: platforms){
        std::vector<cl::Device> devices;
        platform.getDevices(CL_DEVICE_TYPE_CPU, &devices);
        if(!devices.empty()){
            device = devices.front();
            break;
        }
    }

    // Создаем контекст и очередь команд
    cl::Context context(device);
    cl::CommandQueue queue(context, device);

    // Выделяем память на GPU
    cl::Buffer bufferA(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, sizeof(float) * A.size(), A.data());
    cl::Buffer bufferB(context, CL_MEM_READ_ONLY | CL_MEM_COPY_HOST_PTR, sizeof(float) * B.size(), B.data());
    cl::Buffer bufferC(context, CL_MEM_WRITE_ONLY, sizeof(float) * C.size());

    // Компилируем ядро
    const char* kernel_code = R"(
     __kernel void vector_add(__global const float *A, __global const float *B, __global float *C) {
            int id = get_global_id(0);
            C[id] = A[id] + B[id];
        }
    )";
    cl::Program program(context, kernel_code);
    program.build("-cl-std=CL1.2");

    // Создаем ядро и устанавливаем аргументы
    cl::Kernel kernel(program, "vector_add");
    kernel.setArg(0, bufferA);
    kernel.setArg(1, bufferB);
    kernel.setArg(2, bufferC);

    // Запускаем ядро
    queue.enqueueNDRangeKernel(kernel, cl::NullRange, cl::NDRange(A.size()), cl::NullRange);

    // Копируем результат обратно на CPU
    queue.enqueueReadBuffer(bufferC, CL_TRUE, 0, sizeof(float) * C.size(), C.data());

    // Выводим результат
    for (float c : C) {
        std::cout << c << " ";
    }
    return 0;
}



Overwriting example.cpp


In [52]:
!rm -rf build/ && mkdir build && cd build && cmake ../ && cmake --build .

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for CL_VERSION_3_0
-- Looking for CL_VERSION_3_0 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "3.0")
-- Configuring done (0.6s)
-- Generating done (0.0s)
-- Build files have been written to: /content/build
[ 50%] Building CXX object CMakeFiles/examples_ocl.dir/example.cpp.o
In file included from /content/example.cpp:2:
/usr/include/CL/opencl.hpp:437:112: note: ‘#pragma message: opencl.hpp: CL_HPP_TARGET_OPENCL_VERSION is not defined. It will defa

In [53]:
!cd build && ./examples_ocl

Platform Name: Portable Computing Language
Platform Vendor: The pocl project
Platform Version: OpenCL 2.0 pocl 1.8  Linux, None+Asserts, RELOC, LLVM 11.1.0, SLEEF, DISTRO, POCL_DEBUG
Platform Profile: FULL_PROFILE
Platform Extensions: cl_khr_icd cl_pocl_content_size

Device Name: pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
Device Vendor: GenuineIntel
11 22 33 44 55 

In [47]:
!clinfo

Number of platforms                               1
  Platform Name                                   Portable Computing Language
  Platform Vendor                                 The pocl project
  Platform Version                                OpenCL 2.0 pocl 1.8  Linux, None+Asserts, RELOC, LLVM 11.1.0, SLEEF, DISTRO, POCL_DEBUG
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_icd cl_pocl_content_size
  Platform Extensions function suffix             POCL

  Platform Name                                   Portable Computing Language
Number of devices                                 1
  Device Name                                     pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
  Device Vendor                                   GenuineIntel
  Device Vendor ID                                0x8086
  Device Version                                  OpenCL 1.2 pocl HSTR: pthread-x86_64-pc-linux-gnu-broadwell
  Driver Versio